# PART 1

Importing Liberaries

In [1]:
import pandas as pd 
import json
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

Solving environment: done

# All requested packages already installed.



Importing Data

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df=pd.read_html(url,header=0)
df = df[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Dropping the Rows where Borough is 'Not assigned' and converting 'Not assigned' in Neighborhood to similar to Boroughs

In [3]:
df = df[df.Borough != "Not assigned"].reset_index(drop=True)
df.Neighborhood.replace("Not assigned", df.Borough, inplace=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Since already the data provided is grouped by Postal code, not using the groupby method, but replacing the '/' with ',' to divide the Neighborhood

In [4]:
df['Neighborhood']=df.Neighborhood.str.replace('/',',')
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


Finding the shape

In [5]:
df.shape

(103, 3)

# PART 2

Importing the Geographical data

In [6]:
df_geo=pd.read_csv('https://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Adjusting the Column names and merging the 2 databases to one

In [7]:
df_geo.rename(columns={'Postal Code':'Postal code'},inplace=True)
df_canada=pd.merge(df,df_geo,on=['Postal code'])
df_canada.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


Finding the shape

In [8]:
df_canada.shape

(103, 5)

# PART 3 

Selecting only boroughs that contain the word Toronto

In [9]:
Toronto_data = df_canada[df_canada['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Getting a Look at how the datapoints are located on the map 

In [10]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [11]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=15)

for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.8,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Using Foursquare to Explore the Neighborhoods in Toronto

In [12]:
CLIENT_ID = 'BMCPFLRXKSSQSMCOBF3M3NFRELEGS35ACDJYAO1L4KWA2L2H' 
CLIENT_SECRET = 'K2TJGI5D1XMRHEXLFHUDUEGVHC4TH5LGN514BGGBKF1YKLMP' 
VERSION = '20180605'

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude'])

Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst  Quay , South Niagara , Island airport
Rosed

In [14]:
print(toronto_venues.shape)
toronto_venues.head()

(1661, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park , Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [15]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton , Parkdale Village , Exhibition Place",22,22,22,22,22,22
Business reply mail Processing CentrE,17,17,17,17,17,17
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",16,16,16,16,16,16
Central Bay Street,73,73,73,73,73,73
Christie,17,17,17,17,17,17
Church and Wellesley,80,80,80,80,80,80
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


In [16]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


# Analyzing each Neighborhood

In [17]:
Tor_oh = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

Tor_oh['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [Tor_oh.columns[-1]] + list(Tor_oh.columns[:-1])
Tor_oh = Tor_oh[fixed_columns]

print('Tor_oh shape:', Tor_oh.shape)
Tor_oh.head()

Tor_oh shape: (1661, 235)


,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
Tor_group = Tor_oh.groupby('Neighborhood').mean().reset_index()
print('Tor_group shape:', Tor_group.shape)
Tor_group

Tor_group shape: (38, 235)


,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.0000,0.000000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.0000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing CentrE,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.0000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.0000,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.0000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.013699,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.013699,0.000000,...,0.0000,0.000000,0.00000,0.00,0.013699,0.000000,0.000000,0.013699,0.000000,0.000000
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.0000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.025000,0.0125,0.0000,0.0000,0.000,0.0000,0.000,0.012500,0.000000,...,0.0125,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.012500,0.000000
7,"Commerce Court , Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,...,0.0000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.0000,0.028571,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.0000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Checking each neighborhood along with the top 3 most common venues

In [19]:
num_top_venues = 3

for hood in Tor_group['Neighborhood']:
    print("----"+hood+"----")
    temp = Tor_group[Tor_group['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.05
1  Cocktail Bar  0.05
2    Restaurant  0.04


----Brockton , Parkdale Village , Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09


----Business reply mail Processing CentrE----
                  venue  freq
0  Gym / Fitness Center  0.06
1         Garden Center  0.06
2                  Park  0.06


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst  Quay , South Niagara , Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1  Italian Restaurant  0.07
2      Sandwich Place  0.04


----Christie----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.06
1 

Creating cluster Neighborhoods and plotting on the Map for better analysis

In [27]:
import numpy as np

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 11

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhood_venues_sort = pd.DataFrame(columns=columns)
neighborhood_venues_sort['Neighborhood'] = Tor_group['Neighborhood']

for ind in np.arange(Tor_group.shape[0]):
    neighborhood_venues_sort.iloc[ind, 1:] = return_most_common_venues(Tor_group.iloc[ind, :], num_top_venues)

neighborhood_venues_sort.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue
0,Berczy Park,Cocktail Bar,Coffee Shop,Beer Bar,Farmers Market,Café,Cheese Shop,Restaurant,Bakery,Seafood Restaurant,Italian Restaurant,Japanese Restaurant
1,"Brockton , Parkdale Village , Exhibition Place",Café,Breakfast Spot,Coffee Shop,Pet Store,Nightclub,Restaurant,Burrito Place,Italian Restaurant,Intersection,Stadium,Bar
2,Business reply mail Processing CentrE,Restaurant,Comic Shop,Fast Food Restaurant,Farmers Market,Light Rail Station,Burrito Place,Spa,Auto Workshop,Recording Studio,Pizza Place,Butcher
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport Terminal,Rental Car Location,Sculpture Garden,Airport,Airport Food Court,Coffee Shop,Boutique,Boat or Ferry,Harbor / Marina
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Café,Spa,Thai Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Burger Joint,Gym / Fitness Center


In [21]:
kclusters = 5

Tor_grouped_clustering = Tor_group.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Tor_grouped_clustering)

kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [28]:
neighborhood_venues_sort.insert(0, 'ClusterLabels', kmeans.labels_)

Toronto_merge = Toronto_data

Toronto_merge = Toronto_merge.join(neighborhood_venues_sort.set_index('Neighborhood'), on='Neighborhood',how = 'right')
Toronto_merge.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Restaurant,Breakfast Spot,Café,Mexican Restaurant,Ice Cream Shop,Spa,Beer Store
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Yoga Studio,College Cafeteria,Sandwich Place,Restaurant,Burger Joint,Burrito Place,Café,Park,College Auditorium
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Café,Diner,Ramen Restaurant,Bookstore,Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Restaurant,Gastropub,Beer Bar,Hotel,American Restaurant,Italian Restaurant,Department Store,Bakery
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Pub,Trail,Health Food Store,Cuban Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [29]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merge['Latitude'], Toronto_merge['Longitude'], Toronto_merge['Neighborhood'], Toronto_merge['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examining Clusters

In [31]:
Toronto_merge.loc[Toronto_merge['ClusterLabels'] == 0, Toronto_merge.columns[[1] + list(range(5, Toronto_merge.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Bakery,Park,Restaurant,Breakfast Spot,Café,Mexican Restaurant,Ice Cream Shop,Spa,Beer Store
1,Downtown Toronto,0,Coffee Shop,Diner,Yoga Studio,College Cafeteria,Sandwich Place,Restaurant,Burger Joint,Burrito Place,Café,Park,College Auditorium
2,Downtown Toronto,0,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Café,Diner,Ramen Restaurant,Bookstore,Restaurant
3,Downtown Toronto,0,Coffee Shop,Café,Cocktail Bar,Restaurant,Gastropub,Beer Bar,Hotel,American Restaurant,Italian Restaurant,Department Store,Bakery
5,Downtown Toronto,0,Cocktail Bar,Coffee Shop,Beer Bar,Farmers Market,Café,Cheese Shop,Restaurant,Bakery,Seafood Restaurant,Italian Restaurant,Japanese Restaurant
6,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Café,Spa,Thai Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Burger Joint,Gym / Fitness Center
7,Downtown Toronto,0,Grocery Store,Café,Park,Italian Restaurant,Baby Store,Restaurant,Candy Store,Diner,Nightclub,Gas Station,Coffee Shop
8,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Gym,Hotel,Bakery,Thai Restaurant,Cosmetics Shop,Steakhouse,Concert Hall,Salad Place
9,West Toronto,0,Pharmacy,Bakery,Furniture / Home Store,Bank,Supermarket,Café,Middle Eastern Restaurant,Bus Stop,Bar,Gym / Fitness Center,Grocery Store
10,Downtown Toronto,0,Coffee Shop,Aquarium,Italian Restaurant,Café,Hotel,Restaurant,Fried Chicken Joint,Scenic Lookout,Brewery,Sporting Goods Shop,Plaza


In [32]:
Toronto_merge.loc[Toronto_merge['ClusterLabels'] == 1, Toronto_merge.columns[[1] + list(range(5, Toronto_merge.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue
19,Central Toronto,1,Garden,Women's Store,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [33]:
Toronto_merge.loc[Toronto_merge['ClusterLabels'] == 2, Toronto_merge.columns[[1] + list(range(5, Toronto_merge.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue
18,Central Toronto,2,Park,Bus Line,Swim School,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [34]:
Toronto_merge.loc[Toronto_merge['ClusterLabels'] == 3, Toronto_merge.columns[[1] + list(range(5, Toronto_merge.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue
21,Central Toronto,3,Park,Jewelry Store,Trail,Sushi Restaurant,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
33,Downtown Toronto,3,Park,Trail,Playground,Creperie,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


Toronto has good food Joints, the cluster with Food joints as the 1st Most Common venue has more items in the list